In [ ]:
import csv
import pandas as pd
import numpy as np
from random import randrange
myname = "Naitik-Malav"

# Decision Tree Implementation using Gini Index
Decision tree using binary univariate split, gini index and information gain.
### Learn
learn function of class DecisionTree is used to build the decision tree by calling build_decision_tree function.

In [ ]:
class DecisionTree():
    tree = {}
    
    def learn(self, data_set):
        # implement this function
        self.tree = build_decision_tree(data_set)
        return self.tree 

### build_decision_tree
Every Node of the decision tree is a question and based on a question we have two child nodes. 
First we have to calculate the information gain, so I have calculated using gini index.

In [ ]:
def build_decision_tree(data_set):
    question, info_gain = best_split(data_set)

    if info_gain != 0:
        trueRows, falseRows = partitioning(data_set, question)

        #recursive call to each rows
        trueBranch = build_decision_tree(trueRows)
        falseBranch = build_decision_tree(falseRows)

        # returns a Question-Decision node
        return Decision_Node(question, trueBranch, falseBranch)

    
    # 0 information gain means it's a leaf node
    return Leaf(data_set)

### best_split
Iterating over every value and we will calculate the information index and highest information index will given more preference.

In [ ]:
def best_split(data_set):
    
    #best information gain and question of the node
    best_question = None 
    best_information_gain = 0  
    
    uncertainty = gini_index(data_set)
    
    for column in range(len(data_set[0])-1): 
        
        values = set()
        for row in data_set:
            values.add(row[column])
            
        for value in values: 
            question = Question(column, value)
            trueRows, falseRows = partitioning(data_set, question)

            if len(falseRows)==0:
                continue
            if len(trueRows)==0:
                continue

            info_gain = information_gain(trueRows, falseRows, uncertainty)

            if best_information_gain <= info_gain:
                best_information_gain = info_gain
                best_question = question

    return best_question, best_information_gain

### Question
It is used to partition the dataset. 
It stores the column number and column value.  

In [ ]:
class Question:
    def __init__(self, x, y):
        self.column, self.value = x, y

    # Compares the feature value in an example to the feature value in this question.
    def match(self, example):
        x = example[self.column]
        if isinstance(x, int) or isinstance(x, float) == True:
            return x >= self.value
        else:
            return x == self.value

    # used to print the question of the tree
    def __repr__(self):
        return utility(self)

In [ ]:
def utility(Node):
    condition = "=="
    if isinstance(Node.value, int) or isinstance(Node.value, float) == True:
        condition = ">="
    return "Is %s %s %s?" % (header[Node.column], condition, str(Node.value))

### partitioning the dataset
If a row matches with the question then add it to trueRows else add it to falseRows

In [ ]:
def partitioning(data_set, question):
    trueRows = []
    falseRows = []
    
    for row in data_set:
        if question.match(row)==False:
            falseRows.append(row)
        else:
            trueRows.append(row)
    return trueRows, falseRows

### Decision_Node

In [ ]:
class Decision_Node:
    def __init__(self, question, trueBranch, falseBranch):
        self.question, self.true, self.false = question, trueBranch, falseBranch

### Information Gain

In [ ]:
def information_gain(left, right, current):
    
    p = float(len(left)) / (len(left) + len(right))
    return current - (p*gini_index(left) + (1-p)*gini_index(right))

### Gini Index

In [ ]:
def gini_index(data_set):
    labels = Label_Count(data_set)
    impurity = 1
    for label in labels:
        probability = labels[label]/float(len(data_set))
        impurity = impurity - (probability*probability)
    return impurity

# Classification
Classifying the test_set using the decision tree (d_tree)

In [ ]:
def classify(instance, Node):
    #Leaf Node
    if isinstance(Node, Leaf):
        return Node.predictions

    #recursive call to 'true' side and 'false' side
    if Node.question.match(instance):
        return classify(instance, Node.true)
    else:
        return classify(instance, Node.false)

### Leaf Node
A Leaf node holds a dictionary of label as key and number of times it appears in the rows from the training data that reach this leaf. That is data of the subset present at leaf node.

In [ ]:
class Leaf:
    
    def __init__(self, rows):
        self.predictions = Label_Count(rows)

### Label_Count
Count the number of each type of label in a dataset and returns the dictionary of label as a key and count as a value.

In [ ]:
def Label_Count(rows):
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

### 10-Fold Cross Validation function

In [ ]:
def cross_validation_split(data_set, folds=10):
    dataset_split = []
    dataset_copy = list(data_set)
    size = int(len(data_set) / folds)
    
    for i in range(folds):
        fold = list()
        while len(fold) < size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    
    return dataset_split

### utility function


In [ ]:
def run_decision_tree():

    # Loading data set
    with open("wine-dataset.csv") as f:
        next(f, None)
        data = [line for line in csv.reader(f, delimiter=",")]    #data_set is a numpy array
    print("Number of records: %d" % len(data))

    
    f = open(myname+"resultg.txt", "w")
    #10-fold cross validation
#     K = 10
#     training_set = [x for i, x in enumerate(data) if i % K != 9]
#     test_set = [x for i, x in enumerate(data) if i % K == 9]

#     print(test_set)
    
    accuracies = []
    folds = cross_validation_split(data)
    
    for i in range(len(folds)):
        training_set = []
        test_set = []
        for j in range(len(folds)):
            if i==j:
                test_set = folds[j]
            else:
                for k in range(len(folds[j])):
                    training_set.append(folds[j][k])
        
        #print(training_set)
        # Construct a tree using training set
        obj = DecisionTree()
        d_tree = obj.learn( training_set )
        

        # Classify the test set using the tree we just constructed
        # I have implemented classify seperately or you can say I have done classification implementation outside of the class
        results = []     
        for instance in test_set:
            #print("hello", instance[:-1])
            #print(classify(instance[:-1], my_tree ))
            #print(list(classify(instance[:-1], my_tree ).keys())[0])
            result = list((classify(instance[:-1], d_tree)).keys())[0]    #stores the count of (total positive + total negative)
            results.append( result == instance[-1])

        # Calculating Accuracy
        accuracy = float(results.count(True))/float(len(results))
        
        #print("accuracy: %.4f" % accuracy)
        f.write("accuracy: %.4f" % accuracy)
        f.write("\n")
        accuracies.append(accuracy)

        
    avg_accuracy = 0
    
    for x in accuracies:
        avg_accuracy += x
    
    avg_accuracy = float(avg_accuracy / float(len(accuracies)))
    
    print("Average Accuracy: %.4f" % avg_accuracy)
    f.write("Average Accuracy: %.4f" % avg_accuracy)
    f.close()


### main function

In [ ]:
if __name__ == "__main__":
    run_decision_tree()